### Selección del modelo

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score


In [42]:
# Cargar el archivo CSV con los datos procesados en un DataFrame de pandas
data = pd.read_csv('../data/data.csv')

In [43]:
# Separar las características de la variable objetivo en el DataFrame
X,y = data.drop('Estado al egreso', axis=1), data['Estado al egreso']

Se realizó una validación cruzada anidada para comparar de manera justa múltiples modelos con un conjunto de datos muy pequeño (21 muestras). Para cada modelo (Random Forest, Regresión Logística y Árbol de Decisión), se utilizó Leave-One-Out (LOO) como bucle externo: en cada iteración, 20 muestras se usaron para entrenamiento y 1 para prueba.
Dentro del conjunto de entrenamiento, la optimización de hiperparámetros se llevó a cabo mediante GridSearchCV con una validación cruzada interna pequeña (3 particiones).
El mejor modelo obtenido en el bucle interno se evaluó luego en la muestra dejada fuera. Este proceso se repitió 21 veces (una por muestra) para recopilar todas las predicciones. Finalmente, se calcularon las métricas a partir de estas predicciones.

Este método previene el sobreajuste asegurando que:

1. La optimización de hiperparámetros nunca tenga acceso a la muestra de prueba.

2. Cada modelo se evalúe en cada muestra exactamente una vez, bajo condiciones idénticas.

3. Los modelos se mantengan simples mediante la restricción de los espacios de hiperparámetros, evitando así una sobreoptimización en datos limitados.

### Random Forest

In [44]:
from sklearn.ensemble import RandomForestClassifier

# Definición del método Leave-One-Out para validación cruzada externa
loo = LeaveOneOut()

# Inicialización del clasificador Random Forest
rf = RandomForestClassifier(random_state=1, class_weight='balanced')

# Parámetros a explorar en la búsqueda de hiperparámetros
params_rf = {
    'n_estimators': [10, 20, 30],
    'max_depth': [2, 3],          
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [5, 10]
}

rf_preds = [] # Predicciones del modelo para cada muestra de prueba
y_true_rf = [] # Valores reales correspondientes a cada muestra de prueba

# Bucle externo de validación cruzada Leave-One-Out
for train_idx, test_idx in loo.split(X):
    # División del conjunto de datos en entrenamiento y prueba para esta iteración
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Búsqueda interna de hiperparámetros mediante validación cruzada 3-fold
    grid_rf = GridSearchCV(rf, params_rf, cv=3, scoring='f1')
    grid_rf.fit(X_train, y_train)
    
    # Predicción con el mejor modelo encontrado para la muestra de prueba
    y_pred = grid_rf.best_estimator_.predict(X_test)
    # Almacenamiento de la predicción y el valor real para evaluación posterior
    rf_preds.append(y_pred[0])
    y_true_rf.append(y_test.values[0])

# Cálculo de métricas
f1_rf = f1_score(y_true_rf, rf_preds)
precision_rf = precision_score(y_true_rf, rf_preds, zero_division=0)
recall_rf = recall_score(y_true_rf, rf_preds)

print(f"RandomForest F1: {f1_rf:.3f}, Precision: {precision_rf:.3f}, Recall: {recall_rf:.3f}")

RandomForest F1: 0.923, Precision: 0.923, Recall: 0.923


In [45]:
# Obtener el mejor estimador encontrado por GridSearchCV después del ajuste
best_rf = grid_rf.best_estimator_
best_rf

RandomForestClassifier(class_weight='balanced', max_depth=2, min_samples_leaf=3,
                       min_samples_split=10, n_estimators=20, random_state=1)

### Regresión Logística

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score

# Inicialización del clasificador de Regresión Logística
lr = LogisticRegression(random_state=1, max_iter=10000)

# Parámetros a explorar en la búsqueda de hiperparámetros
params_lr = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1], 
    'solver': ['saga'],
}

lr_preds = []  # Predicciones del modelo para cada muestra de prueba
y_true_lr = []  # Valores reales correspondientes a cada muestra de prueba

# Bucle externo de validación cruzada Leave-One-Out
for train_idx, test_idx in loo.split(X):
    # División del conjunto de datos en entrenamiento y prueba para esta iteración
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

    # Búsqueda interna de hiperparámetros mediante validación cruzada 3-fold
    grid_lr = GridSearchCV(lr, params_lr, cv=3, scoring='f1', n_jobs=-1)
    grid_lr.fit(X_train, y_train)

    # Predicción con el mejor modelo encontrado para la muestra de prueba
    y_pred = grid_lr.best_estimator_.predict(X_test)
    # Almacenamiento de la predicción y el valor real para evaluación posterior
    lr_preds.append(y_pred[0])
    y_true_lr.append(y_test.values[0])

# Cálculo de métricas
f1_lr = f1_score(y_true_lr, lr_preds)
precision_lr = precision_score(y_true_lr, lr_preds, zero_division=0)
recall_lr = recall_score(y_true_lr, lr_preds)

print(f"LogisticRegression - F1: {f1_lr:.3f}, Precision: {precision_lr:.3f}, Recall: {recall_lr:.3f}")

LogisticRegression - F1: 0.889, Precision: 0.857, Recall: 0.923


In [47]:
# Obtener el mejor estimador encontrado por GridSearchCV después del ajuste
best_lr = grid_lr.best_estimator_
best_lr

LogisticRegression(C=0.1, max_iter=10000, random_state=1, solver='saga')

### Árbol de Decisión 

In [48]:
from sklearn.tree import DecisionTreeClassifier

# Inicialización del clasificador Árbol de Decisión
dt = DecisionTreeClassifier(random_state=1, class_weight='balanced')

# Parámetros a explorar en la búsqueda de hiperparámetros
params_dt = {
    'max_depth': [2, 3, 6],
    'min_samples_leaf': [2, 4, 6],
    'min_samples_split': [2, 4],
    'criterion': ['gini', 'entropy']
}

dt_preds = []  # Predicciones del modelo para cada muestra de prueba
y_true_dt = []  # Valores reales correspondientes a cada muestra de prueba

# Bucle externo de validación cruzada Leave-One-Out
for train_idx, test_idx in loo.split(X):
    # División del conjunto de datos en entrenamiento y prueba para esta iteración
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

    # Búsqueda interna de hiperparámetros mediante validación cruzada 3-fold
    grid_dt = GridSearchCV(dt, params_dt, cv=3, scoring='f1', n_jobs=-1)
    grid_dt.fit(X_train, y_train)

    # Predicción con el mejor modelo encontrado para la muestra de prueba
    y_pred = grid_dt.best_estimator_.predict(X_test)
    # Almacenamiento de la predicción y el valor real para evaluación posterior
    dt_preds.append(y_pred[0])
    y_true_dt.append(y_test.values[0])

# Cálculo de métricas
f1_dt = f1_score(y_true_dt, dt_preds)
precision_dt = precision_score(y_true_dt, dt_preds, zero_division=0)
recall_dt = recall_score(y_true_dt, dt_preds)

print(f"DecisionTree - F1: {f1_dt:.3f}, Precision: {precision_dt:.3f}, Recall: {recall_dt:.3f}")

DecisionTree - F1: 0.696, Precision: 0.800, Recall: 0.615


In [49]:
# Obtener el mejor estimador encontrado por GridSearchCV después del ajuste
best_dt = grid_dt.best_estimator_
best_dt

DecisionTreeClassifier(class_weight='balanced', max_depth=2, min_samples_leaf=4,
                       random_state=1)

Dado que las métricas del DecisionTreeClassifier fueron inferiores a las de los otros modelos, este fue descartado para análisis posteriores.

In [50]:
import pickle as pkl

# Guardar el mejor modelo Random Forest entrenado en un archivo .pkl para uso futuro
with open('../models/rf.pkl', 'wb') as file:
    pkl.dump(best_rf, file)

# Guardar el mejor modelo de Regresión Logística entrenado en un archivo .pkl para uso futuro
with open('../models/lr.pkl', 'wb') as file:
    pkl.dump(best_lr, file)
